# 📚 2.4 Machine Learning

>**Machine Learning (Aprendizaje Automático)** es una subdisciplina de la inteligencia artificial que permite a las computadoras aprender de forma automática a partir de datos, sin ser programadas explícitamente para realizar una tarea específica. En lugar de seguir instrucciones rígidas, los algoritmos de machine learning identifican patrones, relaciones y estructuras en los datos mediante técnicas estadísticas y matemáticas, y utilizan ese conocimiento adquirido para hacer predicciones, clasificaciones o decisiones sobre nuevos datos. Este proceso implica un ciclo iterativo de entrenamiento, evaluación y ajuste del modelo, con el objetivo de mejorar su rendimiento con la experiencia. El machine learning se aplica en una amplia gama de dominios —desde reconocimiento de voz y visión por computadora hasta recomendaciones personalizadas y diagnóstico médico— y se divide en tres paradigmas principales: aprendizaje supervisado (con etiquetas), no supervisado (sin etiquetas) y por refuerzo (mediante recompensas y acciones). Su poder radica en su capacidad para generalizar a partir de ejemplos y adaptarse a entornos cambiantes, convirtiéndolo en una herramienta fundamental en la era del big data y la automatización inteligente.

## Importar paquetes
Estamos en un nuevo notebook, entonces tenemos que volver a importar los paquetes que usaremos:

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV

# Pipeline
from sklearn.pipeline import Pipeline

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB

# Métricas de evaluación
from sklearn.metrics import accuracy_score


# Para guardar el modelo
import pickle

## Carga de datos
En el notebook anterior guardamos nuestros datos procesados en el directorio `data/`. Carguemos estos datos a DataFrames.

In [2]:
df = pd.read_csv('./data/titanic_procesado.csv')

## División de datos
Recordemos que estamos por implementar algoritmos de aprendizaje supervisado. Esto implica que necesitamos dividir nuestros datos en dos conjuntos: uno para entrenamiento y otro para pruebas. El conjunto de entrenamiento se utiliza para ajustar el modelo, mientras que el conjunto de pruebas se usa para evaluar su rendimiento y asegurarnos de que generaliza bien a datos no vistos. Esta división, conocida como división **entrenamiento-prueba**.

El conjunto de entrenamiento se usará en el proceso de entrenamiento para que nuestro modelo pueda “aprenderse” las respuestas correctas a las predicciones que queremos realizar.

### `train_test_split`

Utilizaremos la función `train_test_split` de Sckit Learn para realizar esta división.

Primero que nada, crearemos un DataFrame que contenga los datos sin la variable objetivo (target) Survived, y el otro contendrá únicamente esta variable objetivo Survived.

In [3]:
X = df.drop(['Survived'], axis=1)
y = df['Survived']
X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1.0,1.0,0.433152,0.125,0.0,0.368146,1.0
1,0.0,0.0,0.579431,0.125,0.0,0.615097,0.0
2,1.0,0.0,0.462346,0.000,0.0,0.438286,1.0
3,0.0,0.0,0.563806,0.125,0.0,0.595112,1.0
4,1.0,1.0,0.563806,0.000,0.0,0.448347,1.0


Notamos que nuestra X ya no contiene Survived.

In [4]:
y.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


---

Hagamos la división entrenamiento-prueba.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train = X_train.values  # Convertir a NumPy array
y_train = y_train.values  # Convertir a NumPy array
X_test = X_test.values    # Convertir a NumPy array
y_test = y_test.values    # Convertir a NumPy array

¿Para qué sirven los parámetros `test_size` y `random_state` en `train_test_split`?

**test_size = 0.2**
- **Propósito:** Define qué porcentaje de los datos se destinará al conjunto de **prueba (test)**
- **Valor 0.2:** Significa que el 20% de los datos será para testing y el 80% restante para entrenamiento
- **Ejemplos de valores:**
  - `test_size = 0.2` → 80% entrenamiento, 20% prueba
  - `test_size = 0.3` → 70% entrenamiento, 30% prueba  
  - `test_size = 100` → 100 registros para prueba (número absoluto)

**random_state = 42**
- **Propósito:** Controla la **reproducibilidad** de la división aleatoria
- **Valor 42:** Es una "semilla" que asegura que la división sea siempre la misma
- **Importancia:** Sin este parámetro, cada vez que ejecutes el código obtendrías una división diferente de los datos

**Ejemplo práctico:**
```python
# Sin random_state - resultados diferentes cada vez
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Con random_state - mismos resultados siempre
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
```

**¿Por qué es importante?**
- **test_size:** Asegura una evaluación justa del modelo con datos no vistos
- **random_state:** Permite que tus experimentos sean reproducibles y comparables

>**Nota:** El número 42 es solo una convención popular (referencia a "Guía del Autoestopista Galáctico"), puedes usar cualquier número entero.

## Selección de modelo
Como pudiste notar, importamos una gran cantidad de modelos de Scikit Learn:

```python
# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
```

Ésta es una de las grandes ventajas de implementar aprendizaje automático con herramientas modernas como Python y Scikit Learn. El código que implementaremos nos permitirá probr todos estos modelos. Eligiremos el mejor modelo no a partir de nuestra intuición, sino con base en las métricas de precisión que resulte de cada uno en su fase de entrenamiento.

### GridSearch
Implementaremos una técnica llamada GridSearch, la cual nos permite encontrar los mejores hiperparámetros para nuestro modelo de aprendizaje automático. GridSearch explora exhaustivamente un conjunto predefinido de valores para cada hiperparámetro, entrenando y evaluando el modelo con cada combinación posible. Al final, selecciona la combinación que proporciona el mejor rendimiento según una métrica de evaluación específica, garantizando así que el modelo esté optimizado para obtener los mejores resultados posibles.

El primer paso para implementar GridSearch es crear un diccionario que contenga todos los modelos que queremos probar y los hiperparámetros que queramos probar en cada uno de estos. 

```python
modelos = {
    'Logistic Regression': {
        'model': LogisticRegression(),
        'params': {
            'model__C': [0.1],
            'model__max_iter': [1000]
        }
    },
    'Support Vector Classifier': {
        'model': SVC(),
        'params': {
            'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'model__C': [0.1, 1, 10]
        }
    },
    'Decision Tree Classifier': {
        'model': DecisionTreeClassifier(),
        'params': {
            'model__splitter': ['best', 'random'],
            'model__max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Random Forest Classifier': {
        'model': RandomForestClassifier(),
        'params': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3, 4],
            'model__max_features': ['auto', 'sqrt', 'log2']
        }
    },
    'Gradient Boosting Classifier': {
        'model': GradientBoostingClassifier(),
        'params': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3, 4]
        }
    },
    'AdaBoost Classifier': {
        'model': AdaBoostClassifier(),
        'params': {
            'model__n_estimators': [10, 100]
        }
    },
    'K-Nearest Neighbors Classifier': {
        'model': KNeighborsClassifier(),
        'params': {
            'model__n_neighbors': [3, 5, 7]
        }
    },
    'XGBoost Classifier': {
        'model': XGBClassifier(),
        'params': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3]
        }
    },
    'LGBM Classifier': {
        'model': LGBMClassifier(),
        'params': {
            'model__n_estimators': [10, 100],
            'model__max_depth': [None, 1, 2, 3],
            'model__learning_rate': [0.1, 0.2, 0.3],
            'model__verbose': [-1]
        }
    },
    'GaussianNB': {
        'model': GaussianNB(),
        'params': {}
    },
    'Naive Bayes Classifier': {
        'model': BernoulliNB(),
        'params': {
            'model__alpha': [0.1, 1.0, 10.0]
        }
    }
}
```

> Realiza una rápida investigación de algunos de estos modelos. Describe sus particularidades y los hiperparámetros que requiere cada uno de ellos.

## Entrenamiento
Ejecutemos el código completo que realizará el ajuste de cada uno de los modelos. Ejecútalo en tu notebook, y posteriormente analizaremos línea por línea.

In [6]:
# Definir los modelos y sus respectivos hiperparámetros para GridSearch
modelos = {
    'Regresión Logística': {
        'modelo': LogisticRegression(),
        'parametros': {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear', 'saga'],
            'max_iter': [100, 500, 1000]
        }
    },
    'Clasificador de Vectores de Soporte': {
        'modelo': SVC(),
        'parametros': {
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'C': [0.1, 1, 10]
        }
    },
    'Clasificador de Árbol de Decisión': {
        'modelo': DecisionTreeClassifier(),
        'parametros': {
            'splitter': ['best', 'random'],
            'max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador de Bosques Aleatorios': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3, 4],
            'max_features': ['sqrt', 'log2', None]
        }
    },
    'Clasificador de Gradient Boosting': {
        'modelo': GradientBoostingClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3, 4]
        }
    },
    'Clasificador AdaBoost': {
        'modelo': AdaBoostClassifier(),
        'parametros': {
            'n_estimators': [10, 100]
        }
    },
    'Clasificador K-Nearest Neighbors': {
        'modelo': KNeighborsClassifier(),
        'parametros': {
            'n_neighbors': [3, 5, 7]
        }
    },
    'Clasificador XGBoost': {
        'modelo': XGBClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3]
        }
    },
    'Clasificador LGBM': {
        'modelo': LGBMClassifier(),
        'parametros': {
            'n_estimators': [10, 100],
            'max_depth': [None, 1, 2, 3],
            'learning_rate': [0.1, 0.2, 0.3],
            'verbose': [-1]
        }
    },
    'GaussianNB': {
        'modelo': GaussianNB(),
        'parametros': {}
    },
    'Clasificador Naive Bayes': {
        'modelo': BernoulliNB(),
        'parametros': {
            'alpha': [0.1, 1.0, 10.0]
        }
    }
}

# Inicializar variables para almacenar los puntajes de los modelos y el mejor estimador
puntajes_modelos = []
mejor_precision = 0
mejor_estimador = None
mejor_modelo = None
estimadores = {}

# Iterar sobre cada modelo y sus hiperparámetros
for nombre, info_modelo in modelos.items():
    # Inicializar GridSearchCV con el modelo y los hiperparámetros
    grid_search = GridSearchCV(
        estimator=info_modelo['modelo'],
        param_grid=info_modelo['parametros'],
        cv=5,
        scoring='accuracy',
        verbose=0,
        n_jobs=-1,
    )

    # Ajustar GridSearchCV con los datos de entrenamiento
    grid_search.fit(X_train, y_train)
    
    # Hacer predicciones con el modelo ajustado
    y_pred = grid_search.predict(X_test)
    
    # Calcular la precisión de las predicciones
    precision = accuracy_score(y_test, y_pred)
    
    # Almacenar los resultados del modelo
    puntajes_modelos.append({
        'Modelo': nombre,
        'Precisión': precision
    })

    estimadores[nombre] = grid_search.best_estimator_
    
    # Actualizar el mejor modelo si la precisión actual es mayor que la mejor precisión encontrada
    if precision > mejor_precision:
        mejor_modelo = nombre
        mejor_precision = precision
        mejor_estimador = grid_search.best_estimator_

# Convertir los resultados a un DataFrame para una mejor visualización
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)

# Imprimir el rendimiento de los modelos de clasificación
print("Rendimiento de los modelos de clasificación")
print(metricas.round(2))

# Imprimir el mejor modelo y su precisión
print('---------------------------------------------------')
print("MEJOR MODELO DE CLASIFICACIÓN")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.2f}")

Rendimiento de los modelos de clasificación
                                 Modelo  Precisión
4     Clasificador de Gradient Boosting       0.83
6      Clasificador K-Nearest Neighbors       0.83
7                  Clasificador XGBoost       0.82
8                     Clasificador LGBM       0.81
3    Clasificador de Bosques Aleatorios       0.80
1   Clasificador de Vectores de Soporte       0.80
2     Clasificador de Árbol de Decisión       0.80
0                   Regresión Logística       0.79
5                 Clasificador AdaBoost       0.79
10             Clasificador Naive Bayes       0.79
9                            GaussianNB       0.76
---------------------------------------------------
MEJOR MODELO DE CLASIFICACIÓN
Modelo: Clasificador de Gradient Boosting
Precisión: 0.83


c:\Users\mel_f\Music\Aprendizaje Supervisado Hybridge\Aprendizaje-Supervisado\nombre_proyecto\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


> No te preocupes si ves algunos warnings en el output. Las advertencias no interfieren con el proceso de entrenamiento.

**Investiga qué representa la métrica de precisión.**
La **precisión** (en inglés, *precision*) es una métrica de evaluación utilizada en problemas de clasificación que mide la proporción de predicciones positivas correctas entre todas las predicciones que el modelo ha etiquetado como positivas. En términos formales, se calcula como el cociente entre el número de **verdaderos positivos (TP)** y la suma de verdaderos positivos más **falsos positivos (FP)**:  
\[
\text{Precisión} = \frac{TP}{TP + FP}
\]  
Esta métrica responde a la pregunta: *"De todas las veces que el modelo dijo 'sí' (positivo), ¿cuántas veces estaba realmente en lo correcto?"*. Es especialmente útil cuando el costo de un falso positivo es alto, como en sistemas de detección de enfermedades o fraudes, donde no queremos alertar falsamente. A diferencia del accuracy (exactitud), la precisión ignora los verdaderos negativos, por lo que es más sensible al desequilibrio de clases y al rendimiento en la clase positiva. Por ejemplo, si un modelo tiene una precisión de 0.83, significa que del 83% de las veces que predijo "positivo", fue correcto. 

---

Aunque no lo creas, aquí termina la fase de entrenamiento. Hemos ajustado 11 modelos diferentes, cada uno con diferentes hiperparámetros, y ahora podemos seleccionar aquél que haya logrado obtener una mayor precisión.

Las últimas líneas del código anterior sirven precisamente para mostrarnos, en un formato muy amigable, el resultado de cada uno de los modelos. Podemos ver que el modelo que alcanzó la precisión más alta es el **Clasificador de Gradient Boosting**.

>Es posible que en tu computadora obtengas otro resultado, sin embargo, lo más probable es que la precisión sea similar a 0.83.

**¿y ahora qué?**

Ya que seleccionamos nuestro modelo, podemos comenzar a hacer inferencia. Antes de pasar a este punto, analicemos una por una las líneas de código de entrenamiento.

## Revisión de código

Lo primero que tenemos es el diccionario de modelos. Cada elemento de este diccionario representa un modelo y los diferentes hiperparámetros que probaremos. 

Hemos repetido esta palabra varias veces, pero no nos hemos detenido a revisar qué es. 

Pregunta a la IA de tu preferencia:

Prompt 💡:
> Explica con detalle qué son los hiperparémetros en el contexto del aprendizaje automático.

>**Los hiperparámetros son como los settings ⚙️ que podemos configurar al momento de entrenar un modelo.**

## Sin GridSearch
Para comprender la búsqueda en cuadrícula (Grid Search) más fácilmente, primero veamos cómo se entrena un modelo individualmente. 

Entrenemos una regresión logística:
```python
# Esto ya lo tenemos importado. Lo ponemos nuevamente nada más de referencia
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# Creamos el modelo de regresión logística
model = LogisticRegression()

# Entrenamos el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realizamos predicciones con el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluamos el modelo usando precisión
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")
```
**Output**
```bash
Precisión del modelo: 0.80
```

**¿Y los hiperparámetros?** Si no especificamos hiperparámetros, se utilizarán los valores por defecto que define Scikit Learn para su modelo LogisticRegression

**¿Cómo sabemos qué hiperparámetros podemos usar?**
Esto lo podemos consultar en la documentación del modelo en cuestión. Googlea scikit learn logistic regression. El primer link (o uno de los primeros) que aparece nos lleva la documentación oficial de la regresión logística de Scikit Learn: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

O bien, podemos preguntarle a GhatGPT o cualquier otra IA:

Prompt 💡:
> ¿Cuáles son los hiperparámetros más comunes que podemos utilizar en una regresión logística de Scikit Learn?

Respuesta 🤖:
> **C**: Controla la regularización del modelo. Un valor más bajo implica una regularización más fuerte (mayor penalización por pesos grandes), mientras que un valor más alto implica una regularización más débil. Por defecto es 1.0.
>
> **penalty**: Especifica el tipo de penalización para la regularización. Los valores posibles son 'l1', 'l2', 'elasticnet', o None. 'l2' es el valor por defecto.
>
> **solver**: El algoritmo utilizado para optimizar la función de coste. Los valores posibles incluyen 'newton-cg', 'lbfgs', 'liblinear', 'saga', y 'adam'. 'lbfgs' es el valor por defecto.
>
> **max_iter**: El número máximo de iteraciones para el algoritmo de optimización. El valor por defecto es 100.
>
> **class_weight**: Permite ajustar el peso de las clases en el entrenamiento. Puede ser None, 'balanced', o un diccionario con pesos específicos. Por defecto es None.
>
> **multi_class**: Define el tipo de problema de clasificación múltiple. Los valores posibles son 'ovr' (one-vs-rest) y 'multinomial'. 'ovr' es el valor por defecto.
>
> **fit_intercept**: Indica si se debe ajustar el intercepto. El valor por defecto es True.
>
> **warm_start**: Permite reutilizar la solución de la llamada anterior para iniciar el siguiente ajuste. El valor por defecto es False.
>
> **tol**: El umbral de tolerancia para el criterio de convergencia. El valor por defecto es 1e-4

Entonces podemos implementar nuestra regresión logística especificando algunos hiperparámetros:
```python
model = LogisticRegression(
    C=0.5,                  # Valor de regularización
    penalty='l2',            # Tipo de penalización (l2 es la regularización Ridge)
    solver='lbfgs',         # Algoritmo de optimización
    max_iter=200,           # Número máximo de iteraciones
    class_weight='balanced' # Ajustar pesos de las clases
)

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo: {accuracy:.2f}")
```
**Output:**
```bash
Precisión del modelo: 0.78
```
(Empeoró 😅😅😅)

Cuando vemos la lista larga de hiperparámetros, y todos los posibles valores que podemos dar a cada uno de ellos, sin duda nos sentimos algo abrumados. Después de todo, ¿cómo vamos a aprendernos todas estas combinaciones?

> **¡Éste es precisamente el problema que GridSearch busca resolver!**

En lugar de probar combinaciones de hiperparámetros uno por uno, implementamos un GridSearch que probará todas las diferentes combinaciones de hiperparámetros que espefiquemos.

Veamos nuevamente el objeto de regresión logística en el diccionario de modelos:
```python
'Regresión Logística': {
    'modelo': LogisticRegression(),
    'parametros': {
        'C': [0.1],
        'max_iter': [1000]
    }
},
```
Podemos ver que aquí no estamos probando diferentes valores de `C` o `max_iter`. Cambiemos esto:
```python
'Regresión Logística': {
    'modelo': LogisticRegression(),
    'parametros': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga'],
        'max_iter': [100, 500, 1000]
    }
}
```

Ahora al momento de ejecutar la búsqueda en cuadrícula, probaremos muchas más combinaciones de hiperparámetros para la regresión logística. Claro, esto implica que el proceso de entrenamiento tardará un poco más. 

### Con GridSearch
Lo que hace GridSearch por detrás de las cortinas es construir un modelo por cada posible combinación de hiperparámetros. Si creamos el objeto de la siguiente manera:
```python
'Regresión Logística': {
    'modelo': LogisticRegression(),
    'parametros': {
        'C': [0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga'],
        'max_iter': [100, 500, 1000]
    }
}
```
literalmente se construirán todos estos objetos:
```bash
LogisticRegression(C=0.01, penalty="l1", solver="liblinear", max_iter=100)
LogisticRegression(C=0.01, penalty="l1", solver="liblinear", max_iter=500)
LogisticRegression(C=0.01, penalty="l1", solver="liblinear", max_iter=1000)
LogisticRegression(C=0.01, penalty="l2", solver="liblinear", max_iter=100)
LogisticRegression(C=0.01, penalty="l2", solver="liblinear", max_iter=500)
LogisticRegression(C=0.01, penalty="l2", solver="liblinear", max_iter=1000)
LogisticRegression(C=0.01, penalty="l2", solver="saga", max_iter=100)
LogisticRegression(C=0.01, penalty="l2", solver="saga", max_iter=500)
LogisticRegression(C=0.01, penalty="l2", solver="saga", max_iter=1000)

LogisticRegression(C=0.1, penalty="l1", solver="liblinear", max_iter=100)
LogisticRegression(C=0.1, penalty="l1", solver="liblinear", max_iter=500)
LogisticRegression(C=0.1, penalty="l1", solver="liblinear", max_iter=1000)
LogisticRegression(C=0.1, penalty="l2", solver="liblinear", max_iter=100)
LogisticRegression(C=0.1, penalty="l2", solver="liblinear", max_iter=500)
LogisticRegression(C=0.1, penalty="l2", solver="liblinear", max_iter=1000)
LogisticRegression(C=0.1, penalty="l2", solver="saga", max_iter=100)
LogisticRegression(C=0.1, penalty="l2", solver="saga", max_iter=500)
LogisticRegression(C=0.1, penalty="l2", solver="saga", max_iter=1000)

LogisticRegression(C=1, penalty="l1", solver="liblinear", max_iter=100)
LogisticRegression(C=1, penalty="l1", solver="liblinear", max_iter=500)
LogisticRegression(C=1, penalty="l1", solver="liblinear", max_iter=1000)
LogisticRegression(C=1, penalty="l2", solver="liblinear", max_iter=100)
LogisticRegression(C=1, penalty="l2", solver="liblinear", max_iter=500)
LogisticRegression(C=1, penalty="l2", solver="liblinear", max_iter=1000)
LogisticRegression(C=1, penalty="l2", solver="saga", max_iter=100)
LogisticRegression(C=1, penalty="l2", solver="saga", max_iter=500)
LogisticRegression(C=1, penalty="l2", solver="saga", max_iter=1000)

LogisticRegression(C=10, penalty="l1", solver="liblinear", max_iter=100)
LogisticRegression(C=10, penalty="l1", solver="liblinear", max_iter=500)
LogisticRegression(C=10, penalty="l1", solver="liblinear", max_iter=1000)
LogisticRegression(C=10, penalty="l2", solver="liblinear", max_iter=100)
LogisticRegression(C=10, penalty="l2", solver="liblinear", max_iter=500)
LogisticRegression(C=10, penalty="l2", solver="liblinear", max_iter=1000)
LogisticRegression(C=10, penalty="l2", solver="saga", max_iter=100)
LogisticRegression(C=10, penalty="l2", solver="saga", max_iter=500)
LogisticRegression(C=10, penalty="l2", solver="saga", max_iter=1000)

LogisticRegression(C=100, penalty="l1", solver="liblinear", max_iter=100)
LogisticRegression(C=100, penalty="l1", solver="liblinear", max_iter=500)
LogisticRegression(C=100, penalty="l1", solver="liblinear", max_iter=1000)
LogisticRegression(C=100, penalty="l2", solver="liblinear", max_iter=100)
LogisticRegression(C=100, penalty="l2", solver="liblinear", max_iter=500)
LogisticRegression(C=100, penalty="l2", solver="liblinear", max_iter=1000)
LogisticRegression(C=100, penalty="l2", solver="saga", max_iter=100)
LogisticRegression(C=100, penalty="l2", solver="saga", max_iter=500)
LogisticRegression(C=100, penalty="l2", solver="saga", max_iter=1000)
```

---

Esto realmente representa la ventaja o el avance de machine learning. La regresión logística se empezó a utilizar en el siglo pasado, por lo que no es una técnica nueva.

Sin embargo, lo que sí es nuevo (además del poder computacional que ahora tenemos disponible) es la capacidad de contar con herramientas avanzadas como las que estamos implementando. Estas herramientas nos permiten probar hipótesis y experimentar con modelos de manera mucho más rápida y eficiente.

### Variables auxiliares
Después de definir el diccionario de modelos e hiperparámetros, creamos unas variables auxiliares que nos servirán para mostrar fácilmente el mejor modelo que resulte de la búsqueda en cuadrícula:
```python
# Inicializar variables para almacenar los puntajes de los modelos y el mejor estimador
puntajes_modelos = []
mejor_precision = 0
mejor_estimador = None
mejor_modelo = None
estimadores = {}
```

### Ajuste de modelos
El proceso de ajuste lo haremos por cada uno de los elementos de nuestro diccionario de modelos. 

Utilizaremos nada más y nada menos que un ciclo `for`.
```python
# Iterar sobre cada modelo y sus hiperparámetros
for nombre, info_modelo in modelos.items():
```
Adentro de este ciclo `for` ajustaremos los modelos utilizando GridSearch
```python
grid_search = GridSearchCV(
    estimator=info_modelo['modelo'],
    param_grid=info_modelo['parametros'],
    cv=5,
    scoring='accuracy',
    verbose=0,
    n_jobs=-1,
)
```

>Los parámetros **`cv`**, **`verbose`** y **`n_jobs`** son configuraciones comunes en herramientas de validación y optimización de modelos en librerías como scikit-learn: **`cv`** define el número de particiones (folds) para la validación cruzada, permitiendo evaluar la estabilidad y generalización del modelo; **`verbose`** controla el nivel de detalle de la información que se imprime durante la ejecución (útil para monitorear el progreso, especialmente en procesos largos); y **`n_jobs`** especifica el número de núcleos de CPU a utilizar en paralelo para acelerar el cómputo — usar `-1` aprovecha todos los núcleos disponibles. Juntos, estos parámetros permiten equilibrar eficiencia, transparencia y rigor estadístico en el entrenamiento y ajuste de modelos.

Una vez creado el objeto `grid_search`, procedemos a ajustar el modelo, y posteriormente haremos predicciones utilizando el conjunto `X_test`. Estos resultados los almacenaremos en la variable `y_pred`. 

Finalmente, para medir la presición del modelo que acabamos de ajustar, compararemos los valores de `y_pred` y `y_test` para “ver a cuántos le atinó el modelo”. 

Los resultados los iremos guardando en una lista llamada `puntajes_modelos`:
```python
# Ajustar GridSearchCV con los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Hacer predicciones con el modelo ajustado
y_pred = grid_search.predict(X_test)

# Calcular la precisión de las predicciones
precision = accuracy_score(y_test, y_pred)

# Almacenar los resultados del modelo
puntajes_modelos.append({
    'Modelo': nombre,
    'Precisión': precision
})

estimadores[nombre] = grid_search.best_estimator_
```
El siguiente bloque lo usaremos para ir guardando el mejor modelo y mejor precisión. De esta forma, cuando termine todo el ciclo de entrenamiento, podremos mostrar los resultados fácilmente:
```python
# Actualizar el mejor modelo si la precisión actual es mayor que la mejor precisión encontrada
if precision > mejor_precision:
    mejor_modelo = nombre
    mejor_precision = precision
    mejor_estimador = grid_search.best_estimator_
```

**¡Listo!**

Con esto termina el bloque de código del `for`

Ahora escribiremos código para mostrar los resultados:
```python
# Convertir los resultados a un DataFrame para una mejor visualización
metricas = pd.DataFrame(puntajes_modelos).sort_values('Precisión', ascending=False)

# Imprimir el rendimiento de los modelos de clasificación
print("Rendimiento de los modelos de clasificación")
print(metricas.round(2))

# Imprimir el mejor modelo y su precisión
print('---------------------------------------------------')
print("MEJOR MODELO DE CLASIFICACIÓN")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.2f}")
```
Esto nos mostrará el siguiente output:
```bash
> Rendimiento de los modelos de clasificación
                                 Modelo  Precisión
4     Clasificador de Gradient Boosting       0.83
6      Clasificador K-Nearest Neighbors       0.83
7                  Clasificador XGBoost       0.82
5                 Clasificador AdaBoost       0.81
8                     Clasificador LGBM       0.81
1   Clasificador de Vectores de Soporte       0.80
2     Clasificador de Árbol de Decisión       0.80
3    Clasificador de Bosques Aleatorios       0.80
0                   Regresión Logística       0.79
10             Clasificador Naive Bayes       0.79
9                            GaussianNB       0.76
---------------------------------------------------
MEJOR MODELO DE CLASIFICACIÓN
Modelo: Clasificador de Gradient Boosting
Precisión: 0.83
```


> **¿Cuál es la diferencia entre las variables `mejor_estimador` y `mejor_modelo`?**
> La diferencia es:
>
> - **mejor_modelo**: Es una variable tipo string que guarda el **nombre** del modelo que obtuvo la mayor precisión (por ejemplo, `"Clasificador de Gradient Boosting"`).
>
> - **mejor_estimador**: Es el **objeto del modelo entrenado** (el mejor estimador ajustado por GridSearchCV), es decir, la instancia del modelo con los hiperparámetros óptimos encontrados, lista para hacer predicciones.

### Inferencia
Terminamos nuestro proceso de entrenamiento. Lo que viene ahora es algo que llamamos “inferencia”. Esto significa que podremos usar nuestro modelo entrenado para predecir datos nuevos que no haya visto el modelo todavía. 

En el contexto de nuestro proyecto, podríamos ya usar nuestro modelo para alimentarle nueva información de pasajeros y predecir si sobrevivió o no. 

En nuestro proceso de entrenamiento, guardamos el mejor estimador en la variable mejor_estimador`. Si vemos el contenido de esta variable en Jupyter, veremos algo así:

```python
mejor_estimador
```

Para predecir nuevos valores, usaremos el método predict , mismo que recibe un numpy array como argumento, y éste debe ser de exactamente las mismas dimensiones que X_train y X_test.

Obtengamos los primeros datos de X_train y de y_train:
```python
X_train[0]
```
```bash
> array([0.        , 1.        , 0.6159084 , 0.        , 0.        ,
       0.55547282, 1.        ])
```
```python
y_train[0]
```
```bash
> np.int64(0) 
```
Ahora creemos un nuevo numpy array con los datos que vemos en `X_train[0]`
```python
nuevos_datos = np.array([0,1,0.6159084,0,0,0.55547282,1]).reshape(1,-1)
```
Y finalmente corramos predict
```python
mejor_estimador.predict(nuevos_datos)
```
```bash
> array([0])
```

> **Nuestro modelo predice que este pasajero no sobrevivió.**

### Guardar el modelo
Este último paso es necesario para la siguiente lección en la que pondremos nuestro modelo en producción. Usaremos un paquete de Python llamado Pickle, el cual nos permite serializar (guardar) objetos de Python en un archivo para luego poder cargarlos y utilizarlos en diferentes entornos, como una API o una aplicación web.

Pickle es particularmente útil cuando queremos guardar modelos entrenados o cualquier otro objeto complejo de Python. Al guardar el pipeline con Pickle, nos aseguramos de que todas las transformaciones de datos y el modelo en sí se conserven tal como fueron entrenados, permitiéndonos hacer predicciones consistentes con nuevos datos en producción sin necesidad de volver a aplicar las mismas transformaciones manualmente.

Si no has importado pickle, hazlo ahora
```python
import pickle
```
Finalmente, guarda el modelo:
```python
with open('modelo.pkl', 'wb') as archivo_estimador:
    pickle.dump(mejor_estimador, archivo_estimador)
```

In [8]:
#  DEFINICIÓN DE MODELOS Y REJILLAS DE HYPER‑PARÁMETROS
modelos = {
    # Regresión logística (solo como referencia)
    "Regresión Logística": {
        "modelo": LogisticRegression(max_iter=500, solver="lbfgs"),
        "parametros": {"C": [0.01, 0.1, 1, 10]},
    },
    # SVM
    "Clasificador SVM": {
        "modelo": SVC(),
        "parametros": {
            "C": [0.1, 1, 10],
            "kernel": ["linear", "rbf"],
            "gamma": ["scale", "auto"],
        },
    },
    # K‑Nearest Neighbours
    "Clasificador K‑Nearest Neighbours": {
        "modelo": KNeighborsClassifier(),
        "parametros": {"n_neighbors": [3, 5, 7, 9], "weights": ["uniform", "distance"]},
    },
    # Árbol de decisión
    "Clasificador de Árbol de Decisión": {
        "modelo": DecisionTreeClassifier(),
        "parametros": {
            "max_depth": [None, 5, 10, 20],
            "min_samples_split": [2, 5, 10],
        },
    },
    # Bosques aleatorios
    "Clasificador de Bosques Aleatorios": {
        "modelo": RandomForestClassifier(),
        "parametros": {
            "n_estimators": [100, 200],
            "max_depth": [None, 10, 20],
            "min_samples_split": [2, 5],
        },
    },
    # Gradient Boosting
    "Clasificador de Gradient Boosting": {
        "modelo": GradientBoostingClassifier(),
        "parametros": {
            "n_estimators": [100, 200],
            "learning_rate": [0.05, 0.1, 0.2],
            "max_depth": [3, 5],
        },
    },
    # AdaBoost
    "Clasificador AdaBoost": {
        "modelo": AdaBoostClassifier(),
        "parametros": {"n_estimators": [50, 100, 200], "learning_rate": [0.5, 1.0, 1.5]},
    },
    # LightGBM
    "Clasificador LGBM": {
        "modelo": LGBMClassifier(),
        "parametros": {
            "n_estimators": [100, 200],
            "learning_rate": [0.05, 0.1],
            "num_leaves": [31, 63],
        },
    },
    # XGBoost
    "Clasificador XGBoost": {
        "modelo": XGBClassifier(use_label_encoder=False, eval_metric="logloss"),
        "parametros": {
            "n_estimators": [100, 200],
            "learning_rate": [0.05, 0.1],
            "max_depth": [3, 5],
        },
    },
    # Naïve Bayes (Gaussian)
    "Clasificador Naive Bayes": {
        "modelo": GaussianNB(),
        "parametros": {},  # sin hiper‑parámetros a buscar
    },
}

#  VARIABLES DE CONTROL
puntajes_modelos = []          # Lista con nombre y precisión de cada modelo
mejor_precision = -np.inf     # Mejor precisión encontrada hasta el momento
mejor_modelo = None           # Nombre del modelo con mejor precisión
mejor_estimador = None        # Instancia del mejor modelo (con mejores hp)

#  BUCLE DE ENTRENAMIENTO + GRID SEARCH
for nombre, info in modelos.items():
    # Configuramos GridSearchCV (si la rejilla está vacía, simplemente usa el modelo tal cual)
    grid_search = GridSearchCV(
        estimator=info["modelo"],
        param_grid=info["parametros"],
        cv=5,
        scoring="accuracy",
        verbose=0,
        n_jobs=-1,
    )

    # Entrenamos
    grid_search.fit(X_train, y_train)

    # Predicción sobre el conjunto de prueba
    y_pred = grid_search.predict(X_test)

    # Métrica de evaluación
    precision = accuracy_score(y_test, y_pred)

    # Guardamos resultados
    puntajes_modelos.append({"Modelo": nombre, "Precisión": precision})

    # Actualizamos el mejor modelo si corresponde
    if precision > mejor_precision:
        mejor_precision = precision
        mejor_modelo = nombre
        mejor_estimador = grid_search.best_estimator_

#  RESUMEN DE RESULTADOS (opcional, pero útil)
metricas = pd.DataFrame(puntajes_modelos).sort_values(
    "Precisión", ascending=False
).reset_index(drop=True)

print("\n=== Rendimiento de los modelos ===")
print(metricas.round(4))

print("\n=== Mejor modelo encontrado ===")
print(f"Modelo: {mejor_modelo}")
print(f"Precisión: {mejor_precision:.4f}")

#  SERIALIZACIÓN DEL MEJOR MODELO
ruta_guardado = "archivo_estimador.pkl"
with open(ruta_guardado, "wb") as f:
    pickle.dump(mejor_estimador, f)

print(f"\n✅ Modelo guardado en: {ruta_guardado}")


=== Rendimiento de los modelos ===
                               Modelo  Precisión
0                   Clasificador LGBM     0.8492
1  Clasificador de Bosques Aleatorios     0.8380
2                Clasificador XGBoost     0.8156
3   Clasificador de Gradient Boosting     0.8045
4   Clasificador K‑Nearest Neighbours     0.7989
5                    Clasificador SVM     0.7989
6               Clasificador AdaBoost     0.7989
7   Clasificador de Árbol de Decisión     0.7989
8                 Regresión Logística     0.7933
9            Clasificador Naive Bayes     0.7598

=== Mejor modelo encontrado ===
Modelo: Clasificador LGBM
Precisión: 0.8492

✅ Modelo guardado en: archivo_estimador.pkl


c:\Users\mel_f\Music\Aprendizaje Supervisado Hybridge\Aprendizaje-Supervisado\nombre_proyecto\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\mel_f\Music\Aprendizaje Supervisado Hybridge\Aprendizaje-Supervisado\nombre_proyecto\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:49:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
